# wps_QDM

#### wps_QDM is a process that runs the [qdm.netcdf.wrapper](https://cran.r-project.org/web/packages/ClimDown/ClimDown.pdf#page=7) function of [ClimDown](https://cran.r-project.org/web/packages/ClimDown/index.html) package. To get started, first instatiate the client. Here, the client will try to connect to a remote chickadee instance using the url parameter.

In [1]:
import os
from birdy import WPSClient
from wps_tools.testing import get_target_url
from pkg_resources import resource_filename
from tempfile import NamedTemporaryFile
from netCDF4 import Dataset
from datetime import date

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "chickadee":
    os.chdir('../')

In [2]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("chickadee")
print(f"Using chickadee on {url}")

Using chickadee on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/chickadee/wps


In [3]:
chickadee = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [4]:
# NBVAL_IGNORE_OUTPUT
chickadee.qdm?

Signature:
chickadee.qdm(
    gcm_file,
    obs_file=None,
    varname=None,
    num_cores='4',
    loglevel='INFO',
    units_bool=True,
    n_pr_bool=True,
    tasmax_units='celsius',
    tasmin_units='celsius',
    pr_units='kg m-2 d-1',
    max_gb=1.0,
    start_date=datetime.date(1971, 1, 1),
    end_date=datetime.date(2005, 12, 31),
    multiyear=True,
    expand_multiyear=True,
    multiyear_window_length='30',
    trace=0.005,
    jitter_factor=0.01,
    pr_tau='1001',
    tasmax_tau='101',
    tasmin_tau='101',
    pr_seasonal=True,
    tasmax_seasonal=False,
    tasmin_seasonal=False,
    pr_ratio=True,
    tasmax_ratio=False,
    tasmin_ratio=False,
    out_file=None,
    output_formats=None,
)
Docstring:
High-level wrapper for Quantile Delta Mapping (QDM)

Parameters
----------
gcm_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of GCM simulations
obs_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`applicati

#### We can use the docstring to ensure we provide the appropriate parameters.

In [5]:
with NamedTemporaryFile(suffix=".nc", prefix="output_", dir="/tmp", delete=True) as out_file:
    output = chickadee.qdm(
        gcm_file = resource_filename("tests", "/data/CI_expected_output.nc"),
        obs_file = resource_filename("tests", "/data/tiny_obs.nc"),
        out_file = out_file.name,
        varname = "tasmax",
        num_cores = 2
    )
# Use asobj=True to access the output file contents as a dataset
output_dataset = output.get(asobj=True)[0]

#### Once the process has completed we can extract the results and ensure it is what we expected.

In [6]:
expected_data = Dataset(resource_filename("tests", "/data/QDM_expected_output.nc"))
for key, value in expected_data.dimensions.items():
    assert str(output_dataset.dimensions[key]) == str(value)